In [29]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4332
load:0x40078000,len:0
load:0x40078000,len:10992
entry 0x4007a6c4
W (64) rtc_clk: Potentially bogus XTAL frequency: 35 MHz, guessing 40 MHz
OSError: [Errno 2] ENOENT
MicroPython v1.9.2-443-g236297f4 on 2017-11-14; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [23]:
%sendtofile --quiet --source ../basicsockets/syslog.py
%sendtofile --quiet --source ../essential_sensor_code/asyncdevices.py

%sendtofile --quiet --source ../wificodes.txt
%sendtofile --quiet --source ../basicsockets/connectwifi.py
%sendtofile --quiet --source ../basicsockets/connecthotspot.py

%sendtofile --quiet --source ../async_web_serve/webserve_funcs.py
%sendtofile --quiet --source ../async_web_serve/websocket_funcs.py

%sendtofile --quiet --source websockettest.html


Sent 34 lines (705 bytes) to syslog.py.
Sent 48 lines (1507 bytes) to asyncdevices.py.
Sent 4 lines (87 bytes) to wificodes.txt.
Sent 34 lines (1042 bytes) to connectwifi.py.
Sent 9 lines (327 bytes) to connecthotspot.py.
Sent 190 lines (6050 bytes) to webserve_funcs.py.
Sent 86 lines (3106 bytes) to websocket_funcs.py.
Sent 47 lines (1243 bytes) to websockettest.html.


In [30]:
#%%writefile plainws_main.py

import time
time.sleep(1.0)

from syslog import log, elog, powid
import os, ustruct, uasyncio
uasyncio.set_debug(0)

from connectwifi import ipnumber
#from connecthotspot import ipnumber

from webserve_funcs import servestaticfile
from webserve_funcs import servefilelist
from webserve_funcs import readhttpheaders
from webserve_funcs import recpostsave
from webserve_funcs import servemessage
from webserve_funcs import convertsavepostfile

from websocket_funcs import make_hybi07_header
from websocket_funcs import servewebsocket


LOG 6 ON 2329
Scanning wifis: DoESLiverpool, DoESDinky, DoESBackup, BV6000
I (16311) network: event 1
LOG 9 Strongest wifi DoESBackup
> auth (b0)
> assoc (0)
> run (10)
I (18461) network: event 4
.
.I (25041) network: GOT_IP
LOG 18 Device has ipnumber 192.168.0.72


In [31]:
#%%writefile --append plainws_main.py

async def wswebsockreply(rpath, writer, wscondition):
    # wscondition = [WScount, 1=go/0=stop, ..(received records).. ]
    incoming = ""
    count = 0
    try:
        while wscondition[1]:
            count += 1
            if len(wscondition) > 2:
                incoming = wscondition.pop(2)
                log("[WS%d incoming] %s"%(wscondition[0], incoming))
                
            if (count%50) == 0:
                x = "count%d" % count
                if incoming:
                    x = "%s;got %s..." % (x,incoming[:5])
                await writer.awrite(make_hybi07_header(len(x)))
                await writer.awrite(x)
            await uasyncio.sleep_ms(100)
            
    except OSError as e:
        elog("[WS%d wpexception] %s"%(wscondition[0], str(e)))
        wscondition[1] = 0
    log("[WS%d wpleave]"%wscondition[0])


In [32]:
#%%writefile --append plainws_main.py

async def handleconnection(reader, writer):
    receivedrequest = await readhttpheaders(reader)
    
    if "POST" in receivedrequest:
        await recpostsave(receivedrequest, reader, "POSTUPLOAD.txt")
        await servemessage(writer, "done")
        await writer.aclose()
        convertsavepostfile("POSTUPLOAD.txt")
        return

    rpath = receivedrequest.get("GET")
    log(receivedrequest)
    
    bcallaclose = True
    if "WebSocketKey" in receivedrequest:
        bcallaclose = await servewebsocket(rpath, reader, writer, receivedrequest["WebSocketKey"], wswebsockreply)
    elif rpath is None:
        pass
    elif rpath == "":
        await servefilelist(writer)
    else:
        await servestaticfile(writer, rpath)
        
    if bcallaclose:
        await writer.aclose()



In [33]:
#%%writefile --append plainws_main.py

# enables rewriting handleconnection function without restarting
def handleconnection_indirect(reader, writer):
    try:
        return handleconnection(reader, writer)
    except OSError as e:
        print("handleconnection exception", str(e))

port = 80
loop = uasyncio.get_event_loop()
print("* Running on http://%s:%s/" % (ipnumber, port))
loop.create_task(uasyncio.start_server(handleconnection_indirect, ipnumber, port))


* Running on http://192.168.0.72:80/


In [34]:
#%%writefile --append plain_main.py

loop.run_forever()


.LOG 24 {'GET': ''} 
LOG 27 {'GET': 'websockettest.html'} 
LOG 27 File 'websockettest.html' (1243 bytes) served in 18ms 
LOG 27 {'WebSocketKey': b'fBgnJbuui/ff7sWGlP34vQ==', 'GET': ''} 
LOG 27 [WS0] b'Hello'
LOG 28 [WS0 incoming] b'Hello' 
LOG 46 [WS0] b'Yoohoo'
LOG 46 [WS0 incoming] b'Yoohoo' 
.........LOG 90 [WS0 close] 
LOG 90 [WS0 rlleave] 
LOG 90 [WS0 wpleave] 
......

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
  File "/lib/uasyncio/core.py", line 77, in run_forever
  File "/lib/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 
